Librerias

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException, NoSuchElementException
import time
from datetime import datetime, timedelta
import pandas as pd
import re
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc

Extraigo primero los enlaces de los torneos.

In [8]:
try:
    if driver is not None:
        driver.quit()
except:
    pass  # Por si driver aún no estaba creado

options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options)
driver.get("https://www.flashscore.com/tennis")

try:
    wait = WebDriverWait(driver, 10)

    # Esperar manualmente a que se resuelva el CAPTCHA
    #input("Resuelve el CAPTCHA (y otras cosas) manualmente en el navegador y presiona Enter aquí para continuar...")

    time.sleep(2)


    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.leftMenu__item")))
    torneos = driver.find_elements(By.CSS_SELECTOR, "div.leftMenu__item")
    enlaces = []

    for torneo in torneos:
        #print("Texto",torneo.text)

        if not "Show more" in torneo.text:
            # Obtén el título del torneo (opcional)
            titulo = torneo.get_attribute("title")
            #print("Título:", titulo)

            # Localiza el enlace (<a>) dentro del elemento "leftMenu_item"
            enlace = torneo.find_element(By.CSS_SELECTOR, "a")
            href = enlace.get_attribute("href")
            if not "calendar" in href and not "rankings" in href and not "camref" in href and not "davis" in href and not "doubles" in href:
                enlaces.append({
                        "href": href,
                        "nombre": titulo
                    })
                #print("Enlace:", href)

except Exception as e:
    print("Error:", e)
    
finally:
    driver.quit()
    

In [57]:
enlaces[38:]

[{'href': 'https://www.flashscore.com/tennis/itf-women-singles/w100-biarritz/',
  'nombre': 'ITF W100 Biarritz Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-gdansk/',
  'nombre': 'ITF W15 Gdansk Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-kayseri-3/',
  'nombre': 'ITF W15 Kayseri 3 Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-kursumlijska-banja-6/',
  'nombre': 'ITF W15 Kursumlijska Banja 6 Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-madrid/',
  'nombre': 'ITF W15 Madrid Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-monastir-23/',
  'nombre': 'ITF W15 Monastir 23 Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-norges-la-ville/',
  'nombre': 'ITF W15 Norges-la-Ville Women'},
 {'href': 'https://www.flashscore.com/tennis/itf-women-singles/w15-san-diego-ca-3/',
  'nombre': 'ITF W15 San Diego, CA 3 Women'},
 {'

In [58]:
try:
    if driver is not None:
        driver.quit()
except:
    pass

options = uc.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")

driver = uc.Chrome(options=options)
driver.get("https://www.flashscore.com/tennis")

try:
    # Configuración de esperas
    wait = WebDriverWait(driver, 15)
    
    # Esperar a que la página cargue
    time.sleep(5)
    print("Página cargada, buscando torneos...")
    
    # Obtener fecha actual para el nombre del archivo
    fecha_actual = datetime.now().strftime("%Y-%m-%d")
    
    # Lista para almacenar todos los datos
    datos = []
    
    # Procesar cada torneo
    for j, e in enumerate(enlaces[38:]):
        href = e['href']
        nombre = e['nombre']
        enlaces_partidos = []
        url = f"{href}results/"
        
        try:
            print(f"\nProcesando torneo {j+1}/{len(enlaces)}: {nombre}")
            driver.get(url)
            time.sleep(3)
            
            try:
                # Esperar y obtener partidos
                wait.until(EC.presence_of_element_located((
                    By.CSS_SELECTOR, ".event__match.event__match--withRowLink.event__match--static.event__match--twoLine"
                )))
                partidos = driver.find_elements(
                    By.CSS_SELECTOR, ".event__match.event__match--withRowLink.event__match--static.event__match--twoLine"
                )
                print(f"Encontrados {len(partidos)} partidos en {nombre}")
            except TimeoutException:
                print(f"No se encontraron partidos en {nombre}")
                continue

            # Procesar cada partido del torneo
            for partido in partidos:
                try:
                    lineas = partido.text.split("\n")
                    enlace = partido.find_element(By.CSS_SELECTOR, "a")
                    url2 = f"{enlace.get_attribute('href')}/match-statistics/0"

                    if len(lineas) >= 5:
                        torneo_actual = {
                            "nombre": nombre,
                            "jugadorA": lineas[1],
                            "jugadorB": lineas[2],
                            "setsA": lineas[3],
                            "setsB": lineas[4],
                            "fecha": lineas[0],
                            "retirado": 'no'
                        }
                        enlaces_partidos.append({
                            "url2": url2,
                            "info_partido": torneo_actual.copy()
                        })
                    else:
                        print(f"Partido con formato inesperado en {nombre}: {lineas}")
                        
                except Exception as e:
                    print(f"Error extrayendo enlace: {e}")
                    continue

            # Procesar estadísticas de cada partido
            for s, partido_data in enumerate(enlaces_partidos):
                url2 = partido_data['url2']
                torneo_info = partido_data['info_partido']
                print(f"\nProcesando partido {s+1}/{len(enlaces_partidos)} de {nombre}")
                
                try:
                    driver.get(url2)
                    time.sleep(3)
                    
                    for intento in range(3):  # Máximo 3 intentos
                        try:
                            # Esperar y obtener secciones de estadísticas
                            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "section")))
                            estadisticas = driver.find_elements(By.CLASS_NAME, "section")
                            time.sleep(2)
                            
                            # Inicializar diccionario de estadísticas
                            stats = {
                                "torneo": torneo_info.get("nombre", ""),
                                "jugadorA": torneo_info.get("jugadorA", ""),
                                "jugadorB": torneo_info.get("jugadorB", ""),
                                "setsA": torneo_info.get("setsA", ""),
                                "setsB": torneo_info.get("setsB", ""),
                                "fecha": torneo_info.get("fecha", ""),
                                "retirado": torneo_info.get("retirado", ""),
                                # Inicializar todas las estadísticas vacías
                                "acesA": "", "acesB": "",
                                "dobleFaltasA": "", "dobleFaltasB": "",
                                # ... (añade aquí el resto de variables estadísticas)
                                "porcentajePrimerSaqueA":"",
                                "porcentajePrimerSaqueB":"",

                                "porcentajePuntosGanadosConPrimerSaqueA":"",
                                "porcentajePuntosGanadosConPrimerSaqueB":"",
                                "puntosGanadosConPrimerSaqueA":"",
                                "puntosGanadosConPrimerSaqueB":"",

                                "porcentajePuntosGanadosConSegundoSaqueA":'',
                                "porcentajePuntosGanadosConSegundoSaqueB":'',
                                "puntosGanadosConSegundoSaqueA":'',
                                "puntosGanadosConSegundoSaqueB":'',

                                "porcentajeBreakPointsSalvadosA":'',
                                "porcentajeBreakPointsSalvadosB":'',
                                "breakPointsSalvadosA":'',
                                "breakPointsSalvadosB":'',

                                "porcentajeBreakPointsA":'',
                                "porcentajeBreakPointsB":'',
                                "breakPointsA":'',
                                "breakPointsB":'',

                                "velocidadMediaPrimerSaqueA":'',
                                "velocidadMediaPrimerSaqueB":'',
                                "velocidadMediaSegundoSaqueA":'',
                                "velocidadMediaSegundoSaqueB":'',

                                "porcentajePuntosGanadosAlRestoConPrimerSaqueA": '',
                                "porcentajePuntosGanadosAlRestoConPrimerSaqueB": '',
                                "puntosGanadosAlRestoConPrimerSaqueA": '',
                                "puntosGanadosAlRestoConPrimerSaqueB": '',

                                "porcentajePuntosGanadosAlRestoConSegundoSaqueA": '',
                                "porcentajePuntosGanadosAlRestoConSegundoSaqueB": '',
                                "puntosGanadosAlRestoConSegundoSaqueA": '',
                                "puntosGanadosAlRestoConSegundoSaqueB": '',

                                "winnersA":'',
                                "winnersB":'',

                                "erroresNoForzadosA":'',
                                "erroresNoForzadosB":'',

                                "porcentajePuntosGanadosRedA":'',
                                "porcentajePuntosGanadosRedB":'',
                                "puntosGanadosRedA":'',
                                "puntosGanadosRedB":'',

                                "porcentajePuntosGanadosSaqueA":'',
                                "porcentajePuntosGanadosSaqueB":'',
                                "puntosGanadosSaqueA":'',
                                "puntosGanadosSaqueB":'',

                                "porcentajePuntosGanadosRestoA":'',
                                "porcentajePuntosGanadosRestoB":'',
                                "puntosGanadosRestoA":'',
                                "puntosGanadosRestoB":'',

                                "porcentajePuntosGanadosA":'',
                                "porcentajePuntosGanadosB":'',
                                "puntosGanadosA":'',
                                "puntosGanadosB":'',

                                "puntosGanadosUltimas10BolasA":'',
                                "puntosGanadosUltimas10BolasB":'',

                                "puntosDePartidoSalvadosA":'',
                                "puntosDePartidoSalvadosB":'',

                                "porcentajeJuegosGanadosSaqueA":'',
                                "porcentajeJuegosGanadosSaqueB":'',
                                "juegosGanadosSaqueA":'',
                                "juegosGanadosSaqueB":'',

                                "porcentajeJuegosGanadosRestoA":'',
                                "porcentajeJuegosGanadosRestoB":'',
                                "juegosGanadosRestoA":'',
                                "juegosGanadosRestoB":'',

                                "porcentajeJuegosGanadosA":'',
                                "porcentajeJuegosGanadosB":'',
                                "juegosGanadosA":'',
                                "juegosGanadosB":'',

                                "cuotaA":'',
                                "cuotaB":'',

                                "cuotaA_2":'',
                                "cuotaB_2":''
                            }
                            
                            # Procesar cada sección de estadísticas
                            for est in estadisticas:
                                lineas = est.text.split("\n")
                                for i, line in enumerate(lineas):
                                    l = line.lower()
                                    if "aces" in l:
                                        stats["acesA"] = lineas[i - 1] if i > 0 else ""
                                        stats["acesB"] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "double faults" in l:
                                        stats['dobleA'] = lineas[i - 1] if i > 0 else ""
                                        stats['dobleB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "1st serve percentage" in l:
                                        stats['porcentajePrimerSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePrimerSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "1st serve points won" in l:
                                        stats['porcentajeJuegosGanadosPrimerSaqueA'] = lineas[i - 2] if i > 1 else ""
                                        stats['juegosGanadosPrimerSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajeJuegosGanadosPrimerSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['juegosGanadosPrimerSaqueB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "2nd serve points won" in l:
                                        stats['porcentajeJuegosGanadosSegundoSaqueA'] = lineas[i - 2] if i > 1 else ""
                                        stats['JuegosGanadosSegundoSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajeJuegosGanadosSegundoSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['JuegosGanadosSegundoSaqueB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "break points saved" in l:
                                        stats['porcentajeBreakPointsSalvadosA'] = lineas[i - 2] if i > 1 else ""
                                        stats['BreakPointsSalvadosA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajeBreakPointsSalvadosB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['BreakPointsSalvadosB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "average 1st serve speed" in l:
                                        stats['velocidadMediaPrimerSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['velocidadMediaPrimerSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "average 2nd serve speed" in l:
                                        stats['velocidadMediaSegundoSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['velocidadMediaSegundoSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "1st return points won" in l:
                                        stats['porcentajePuntosGanadosAlRestoConPrimerSaqueA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosGanadosAlRestoConPrimerSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosGanadosAlRestoConPrimerSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosGanadosAlRestoConPrimerSaqueB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "2nd return points won" in l:
                                        stats['porcentajePuntosGanadosAlRestoConSegundoSaqueA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosGanadosAlRestoConSegundoSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosGanadosAlRestoConSegundoSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosGanadosAlRestoConSegundoSaqueB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "break points converted" in l:
                                        stats['porcentajePuntosBreakConvertidosA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosBreakConvertidosA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosBreakConvertidosB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosBreakConvertidosB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "winners" in l:
                                        stats['winnersA'] = lineas[i - 1] if i > 0 else ""
                                        stats['winnersB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "unforced errors" in l:
                                        stats['erroresNoForzadosA'] = lineas[i - 1] if i > 0 else ""
                                        stats['erroresNoForzadosB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "net points won" in l:
                                        stats['porcentajePuntosGanadosRedA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosGanadosRedA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosGanadosRedB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosGanadosRedB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "service points won" in l:
                                        stats['porcentajePuntosGanadosAlSaqueA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosGanadosAlSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosGanadosAlSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosGanadosAlSaqueB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "return points won" in l:
                                        stats['porcentajePuntosGanadosAlRestoA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosGanadosAlRestoA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosGanadosAlRestoB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosGanadosAlRestoB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "total points won" in l:
                                        stats['porcentajePuntosGanadosA'] = lineas[i - 2] if i > 1 else ""
                                        stats['PuntosGanadosA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajePuntosGanadosB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['PuntosGanadosB'] = lineas[i + 2] if i+2< len(lineas) else ""
                                    if "last 10 balls" in l:
                                        stats['PuntosGanadosUltimas10BolasA'] = lineas[i - 1] if i > 0 else ""
                                        stats['PuntosGanadosUltimas10BolasB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "match points saved" in l:
                                        stats['BolasDePartidoSalvadasA'] = lineas[i - 1] if i > 0 else ""
                                        stats['BolasDePartidoSalvadasB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                    if "service games won" in l:
                                        stats['porcentajeJuegosGanadosAlSaqueA'] = lineas[i - 2] if i > 1 else ""
                                        stats['JuegosGanadosAlSaqueA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajeJuegosGanadosAlSaqueB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['JuegosGanadosAlSaqueB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "return games won" in l:
                                        stats['porcentajeJuegosGanadosAlRestoA'] = lineas[i - 2] if i > 1 else ""
                                        stats['JuegosGanadosAlRestoA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajeJuegosGanadosAlRestoB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['JuegosGanadosAlRestoB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "total games won" in l:
                                        stats['porcentajeJuegosGanadosA'] = lineas[i - 2] if i > 1 else ""
                                        stats['JuegosGanadosA'] = lineas[i - 1] if i > 0 else ""
                                        stats['porcentajeJuegosGanadosB'] = lineas[i + 1] if i+1 < len(lineas) else ""
                                        stats['JuegosGanadosB'] = lineas[i + 2] if i+2 < len(lineas) else ""
                                    if "odds" in l:
                                        if len(lineas) >= i + 7:
                                            stats['cuotaA'] = lineas[i + 6] if i+6 < len(lineas) else ""
                                            stats['cuotaB'] = lineas[i + 7] if i+7 < len(lineas) else ""
                                    if "show more" in l:
                                            stats['cuotaA_2'] = lineas[i - 2] if i > 1 else ""
                                            stats['cuotaB_2'] = lineas[i - 1] if i > 0 else ""
                            
                            # Guardar datos exitosamente recolectados
                            datos.append(stats)
                            print(f"Estadísticas recolectadas para {torneo_info['jugadorA']} vs {torneo_info['jugadorB']}")
                            break  # Salir del bucle de reintentos si tuvo éxito
                            
                        except StaleElementReferenceException:
                            print(f"Intento {intento + 1} fallido (elemento obsoleto), reintentando...")
                            time.sleep(2)
                            if intento == 2:
                                raise
                        except Exception as e:
                            print(f"Error en intento {intento + 1}: {e}")
                            time.sleep(2)
                            if intento == 2:
                                raise
                                
                except Exception as e:
                    print(f"Error procesando partido {s+1}: {e}")
                    continue
                    
        except Exception as e:
            print(f"Error procesando torneo {nombre}: {e}")
            continue

except Exception as e:
    print(f"Error general en el scraping: {e}")
    
finally:
    # Guardar datos y cerrar driver
    driver.quit()
    print("\nScraping completado. Guardando datos...")
    
    if datos:
        df = pd.DataFrame(datos)
        #nombre_archivo = f"partidos_tenis_{fecha_actual}.csv"
        #df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
        print(f"Datos guardados")
        display(df.head())
    else:
        print("No se recolectaron datos para guardar")

Página cargada, buscando torneos...

Procesando torneo 1/52: ITF W100 Biarritz Women
Encontrados 3 partidos en ITF W100 Biarritz Women

Procesando partido 1/3 de ITF W100 Biarritz Women
Estadísticas recolectadas para Hontama M. vs Rame A.

Procesando partido 2/3 de ITF W100 Biarritz Women
Estadísticas recolectadas para Bolsova Zadoinov A. vs Iliev S.

Procesando partido 3/3 de ITF W100 Biarritz Women
Estadísticas recolectadas para Davis L. vs Inisan E.

Procesando torneo 2/52: ITF W15 Gdansk Women
Encontrados 7 partidos en ITF W15 Gdansk Women

Procesando partido 1/7 de ITF W15 Gdansk Women
Estadísticas recolectadas para Affelt N. vs Meyer Auf Der Heide L.

Procesando partido 2/7 de ITF W15 Gdansk Women
Estadísticas recolectadas para Haavisto E. vs Milatova S.

Procesando partido 3/7 de ITF W15 Gdansk Women
Estadísticas recolectadas para Ivanov V. vs Krajmer K.

Procesando partido 4/7 de ITF W15 Gdansk Women
Estadísticas recolectadas para Drugdova S. vs Vasilyeva A.

Procesando partido

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,dobleFaltasA,...,juegosGanadosRestoA,juegosGanadosRestoB,porcentajeJuegosGanadosA,porcentajeJuegosGanadosB,juegosGanadosA,juegosGanadosB,cuotaA,cuotaB,cuotaA_2,cuotaB_2
0,ITF W100 Biarritz Women,Hontama M.,Rame A.,0,2,11.06. 12:10,no,,,,...,,,,,,,1.73,2.00,1.73,2.00
1,ITF W100 Biarritz Women,Bolsova Zadoinov A.,Iliev S.,2,0,11.06. 12:00,no,,,,...,,,,,,,1.14,5.50,1.14,5.50
2,ITF W100 Biarritz Women,Davis L.,Inisan E.,0,1,10.06. 18:00,no,,,,...,,,,,,,Show more,Dobla tu depósito hasta 200€,1.05,9.00
3,ITF W15 Gdansk Women,Affelt N.,Meyer Auf Der Heide L.,0,2,11.06. 12:20,no,,,,...,,,,,,,,,,
4,ITF W15 Gdansk Women,Haavisto E.,Milatova S.,0,2,11.06. 11:30,no,,,,...,,,,,,,3.50,1.29,3.50,1.29


Guardar el dataframe si está bien

In [60]:
df.to_csv(".\\data\\biarritz-ceskalipa_fem_2025-06-11.csv", index=False, encoding='utf-8-sig')

df['torneo'].value_counts()

torneo
ITF W15 Kayseri 3 Women               16
ITF W15 Kursumlijska Banja 6 Women    16
ITF W35 Santo Domingo 4 Women         16
ITF W35 Luzhou Women                  16
ITF W15 Madrid Women                  15
ITF W75 Ceska Lipa Women              14
ITF W15 Tashkent 2 Women              13
ITF W15 Norges-la-Ville Women         12
ITF W15 Monastir 23 Women             11
ITF W50 Guimaraes Women               10
ITF W15 Gdansk Women                   7
ITF W15 San Diego, CA 3 Women          4
ITF W100 Biarritz Women                3
Name: count, dtype: int64